In [1]:
from collections import defaultdict, Counter
import numpy as np
import json
import os
from datetime import datetime
import re
from time import sleep

In [2]:
import spacy
import nltk

In [3]:
from io import StringIO, BytesIO
import pickle
import torch

In [4]:
# Wrapper RNNTagger_German.py muss im gleichen Verzeichnis wie dieses Notebook sein
from RNNTagger_German import RNNTagger_German

In [5]:
from spacy.lang.de.stop_words import STOP_WORDS
more_stopwords = ('tagesschau','willkommen','untertitelung','ndr','studio','abend','fernsehen','copyright','untertitel','schön')
STOP_WORDS.update(more_stopwords)

In [6]:
nlp = spacy.load("de_core_news_lg")

In [7]:
rnntagger_path = "./RNNTagger-1.2.1/RNNTagger/" # Pfad zum RNNTagger (https://www.cis.uni-muenchen.de/~schmid/tools/RNNTagger/)
tagger = RNNTagger_German(rnntagger_path)

In [8]:
TAGESSCHAU_PATH = "C:/Users/Aron/Documents/SoSe_2021/TextViz/Tagesschau/Data"

# Save filenames per month


In [9]:
MONTH_PATTERN = re.compile("Tagesschau_\d?\d-(\d\d-20\d\d)\.json")
handled = 0

months = defaultdict(list)
for file in os.listdir(TAGESSCHAU_PATH):
    if file.endswith(".json") and (match := MONTH_PATTERN.search(file)):
            months[match.group(1)].append(file)
            handled += 1
with open("C:/Users/Aron/Documents/SoSe_2021/TextViz/Tagesschau/month_files.json", 'w', encoding='utf-8') as f:
    json.dump(months, f)

In [9]:
with open("C:/Users/Aron/Documents/SoSe_2021/TextViz/Tagesschau/month_files.json", 'r', encoding='utf-8') as f:
    months = json.load(f)

# Process Tokens per Month

In [11]:
target_dir = "C:/Users/Aron/Documents/SoSe_2021/TextViz/Tagesschau/Months_Filtered/"

In [ ]:
disabled_pipeline = ('ner', 'attribute_ruler', 'lemmatizer')
token_count = 0
time = datetime.now()
relax = 15
tagset = ("NN", "NE", "FM", "ADJD", "ADJA", "VVINF")

for month, files in months.items():
    if f"{month}.json" in os.listdir(target_dir):
        print(f"{month} already tokenized")
        continue
        
    print(f"Start tokenize {month}")
    month_tokens = []
    for file in files:
        time = datetime.now()
        with open(os.path.join(TAGESSCHAU_PATH, file), 'r', encoding='utf-8') as f:
            data = json.load(f)

        txt = data['transcript'].replace('\n\n', ' ').replace('\n', ' ')

        doc = nlp(txt, disable=disabled_pipeline)
        tokens = tagger.process_tokens(doc.sents, filter_tags=tagset, stopwords=STOP_WORDS)
        month_tokens += tokens
        print(f"    Got {len(tokens)} tokens for {file} in {(datetime.now()-time).seconds} seconds")
        print(f"        Cool down CPU for {relax} seconds")
        
        sleep(relax)
        
    
    with open(os.path.join(target_dir, f"{month}.json"), 'w', encoding='utf-8') as f:
        json.dump(month_tokens, f)
    token_count += len(month_tokens)
    print(f"Finished with {month}, {len(month_tokens)} tokens, {token_count} tokens so far")
    

    #break